## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-22-16-33-16 +0000,bbc,Canada Post and union reach deal in principle ...,https://www.bbc.com/news/articles/cvgkeyl78rwo...
1,2025-11-22-16-33-03 +0000,nypost,Ponzi schemer whose sentence got commuted by T...,https://nypost.com/2025/11/22/us-news/ponzi-sc...
2,2025-11-22-16-29-18 +0000,nypost,Carjacking victim’s daughter says New Orleans ...,https://nypost.com/2025/11/22/us-news/carjacki...
3,2025-11-22-16-28-18 +0000,bbc,Three arrested in India after police crack $80...,https://www.bbc.com/news/articles/cy8j4m45yx2o...
4,2025-11-22-16-27-59 +0000,nypost,‘Antisemitic’ activist Linda Sarsour: Mamdani ...,https://nypost.com/2025/11/22/us-news/antisemi...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2329/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
14,trump,55
135,ukraine,19
85,he,18
139,plan,17
230,peace,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
163,2025-11-22-01-00-00 +0000,wsj,President Trump said this week he expects lowe...,https://www.wsj.com/economy/central-banking/fe...,167
29,2025-11-22-15-11-00 +0000,wsj,President Trump said Friday he was immediately...,https://www.wsj.com/politics/policy/trump-says...,133
248,2025-11-21-20-01-54 +0000,nyt,European Leaders Back Zelensky After Trump’s R...,https://www.nytimes.com/2025/11/21/world/europ...,129
218,2025-11-21-21-34-00 +0000,wsj,Trump Says He Wants Ukraine’s Answer on Peace ...,https://www.wsj.com/world/zelensky-russia-ukra...,128
190,2025-11-21-22-54-21 +0000,nyt,"As Trump Pushes on Russia-Ukraine Peace Plan, ...",https://www.nytimes.com/2025/11/21/world/europ...,124


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
163,167,2025-11-22-01-00-00 +0000,wsj,President Trump said this week he expects lowe...,https://www.wsj.com/economy/central-banking/fe...
135,79,2025-11-22-04-00-00 +0000,wsj,A Texas court upended a GOP plan that would he...,https://www.wsj.com/politics/elections/the-leg...
238,57,2025-11-21-20-26-00 +0000,wsj,Oil Futures Fall on U.S. Push for Russia-Ukrai...,https://www.wsj.com/finance/commodities-future...
15,49,2025-11-22-15-51-00 +0000,wsj,"Republican Marjorie Taylor Greene, who fought ...",https://www.wsj.com/politics/policy/marjorie-t...
29,46,2025-11-22-15-11-00 +0000,wsj,President Trump said Friday he was immediately...,https://www.wsj.com/politics/policy/trump-says...
220,44,2025-11-21-21-33-52 +0000,nypost,Trump says Zohran Mamdani is not a ‘jihadist’ ...,https://nypost.com/2025/11/21/us-news/trump-br...
33,42,2025-11-22-15-02-26 +0000,nypost,Ryan Lizza reveals raunchy ‘poem’ RFK Jr alleg...,https://nypost.com/2025/11/22/us-news/ryan-liz...
0,34,2025-11-22-16-33-16 +0000,bbc,Canada Post and union reach deal in principle ...,https://www.bbc.com/news/articles/cvgkeyl78rwo...
250,33,2025-11-21-19-35-34 +0000,latimes,Suspect in car chase crash that killed an Alha...,https://www.latimes.com/california/story/2025-...
283,33,2025-11-21-17-45-27 +0000,nypost,"Grizzly bear attacks students, hero teachers w...",https://nypost.com/2025/11/21/world-news/grizz...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
